In [20]:
import torch
from spikingjelly.datasets import dvs128_gesture
import time
import sys

sys.path.append('./..')

import deepModelsDVS, utils


In [21]:
def load_data(dataset_dir, distributed, T):
    # Data loading code
    print("Loading data")

    st = time.time()

    dataset_train = dvs128_gesture.DVS128Gesture(root=dataset_dir, train=True, data_type='frame', frames_number=T, split_by='number')
    dataset_test = dvs128_gesture.DVS128Gesture(root=dataset_dir, train=False, data_type='frame', frames_number=T, split_by='number')


    print("Took", time.time() - st)

    print("Creating data loaders")
    if distributed:
        train_sampler = torch.utils.data.distributed.DistributedSampler(dataset_train)
        test_sampler = torch.utils.data.distributed.DistributedSampler(dataset_test)
    else:
        train_sampler = torch.utils.data.RandomSampler(dataset_train)
        test_sampler = torch.utils.data.SequentialSampler(dataset_test)

    return dataset_train, dataset_test, train_sampler, test_sampler


In [22]:
data_path = './../datasets/DVSGesture/'
distributed = False
T = 16

dataset_train, dataset_test, train_sampler, test_sampler = load_data(data_path, distributed, T)
print(f'dataset_train:{dataset_train.__len__()}, dataset_test:{dataset_test.__len__()}')

data_loader = torch.utils.data.DataLoader(
    dataset_train, batch_size= 1,
    sampler=train_sampler, num_workers= 8, pin_memory=True)

data_loader_test = torch.utils.data.DataLoader(
    dataset_test, batch_size= 1,
    sampler=test_sampler, num_workers= 8, pin_memory=True)


Loading data
The directory [./../datasets/DVSGesture/frames_number_16_split_by_number] already exists.
The directory [./../datasets/DVSGesture/frames_number_16_split_by_number] already exists.
Took 0.007129192352294922
Creating data loaders
dataset_train:1176, dataset_test:288


In [15]:
device = 'cuda:2'
aModel = 'SEWResNet'
aConnect_f = 'ADD'

model = deepModelsDVS.__dict__[aModel](aConnect_f) #deepModelsDVS.SEWResNet(args.connect_f) 
print("Creating model")

model.to(device)

Creating model


ResNetN(
  (conv): Sequential(
    (0): Sequential(
      (0): SeqToANNContainer(
        (0): Conv2d(2, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): ParametricLIFNode(
        v_threshold=1.0, v_reset=0.0, detach_reset=True, step_mode=s, backend=torch, tau=2.0
        (surrogate_function): Sigmoid(alpha=4.0, spiking=True)
      )
    )
    (1): SEWBlock(
      (conv): Sequential(
        (0): Sequential(
          (0): SeqToANNContainer(
            (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
            (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          )
          (1): ParametricLIFNode(
            v_threshold=1.0, v_reset=0.0, detach_reset=True, step_mode=s, backend=torch, tau=2.0
            (surrogate_function): Sigmoid(alpha=4.0, spiking=True)
          )
        )
        (1): Se

In [23]:
data_loader

In [24]:
model.train()
metric_logger = utils.MetricLogger(delimiter="  ")
metric_logger.add_meter('lr', utils.SmoothedValue(window_size=1, fmt='{value}'))
metric_logger.add_meter('img/s', utils.SmoothedValue(window_size=10, fmt='{value}'))
header = 'Epoch: [{}]'.format(0)
for image, target in metric_logger.log_every(data_loader, 64, header):
    start_time = time.time()
    image, target = image.to(device), target.to(device)
    image = image.float()  # [N, T, C, H, W]
    output = model(image)
    print(output)
    print(target)
    break


tensor([[-0.9257,  5.1297, -2.7845, -3.3966,  0.1226, -1.5026,  0.5724, -0.8258,
          3.4552,  0.3725,  1.7478]], device='cuda:2',
       grad_fn=<AddmmBackward0>)
tensor([0], device='cuda:2')


In [7]:
for image, label in data_loader:
    image, label = image.to('cuda:2'), label.to('cuda:2')
    break

label

tensor([7], device='cuda:2')